In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('questions.csv')

In [5]:
df.shape

(404351, 6)

In [7]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [15]:
new_df = df.sample(30000)

In [17]:
new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [19]:
new_df.duplicated().sum()

0

In [21]:
ques_df = new_df[['question1', 'question2']]
ques_df.head()

,question1,question2
1229,"My parents give me ""advice"" and they get reall...",What is the best advice for giving others an a...
368273,How can I watch any clip4sale video without pa...,Can I use an old game that is now free in a vi...
228170,How can I learn seo?,How do I learn SEO basic?
166591,What are the advantages and disadvantages of D...,What are the disadvantages of using a DBMS?
396394,What is causing universe to expand?,What is causing our universe to expand and acc...


In [23]:
from sklearn.feature_extraction.text import CountVectorizer

In [25]:
# merge texts 
questions = list(ques_df['question1']) + list(ques_df['question2'])

cv = CountVectorizer(max_features=3000)
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(), 2)

In [29]:
temp_df1 = pd.DataFrame(q1_arr, index= ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index=ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape

(30000, 6000)

In [31]:
temp_df['is_duplicate'] = new_df['is_duplicate']

In [33]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,is_duplicate
1229,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
368273,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
228170,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
166591,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
396394,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train , y_test = train_test_split(temp_df.iloc[:, 0:-1].values, temp_df.iloc[:, -1].values, test_size=0.2, random_state=21)

In [39]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [43]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test, y_pred)

0.7465

In [51]:
!pip install xgboost

   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB 217.9 kB/s eta 0:09:34
   ---------------------------------------- 0.0/124.9 MB 217.9 kB/s eta 0:09:34
   ---------------------------------------- 0.1/124.9 MB 297.7 kB/s eta 0:07:00
   ---------------------------------------- 0.1/124.9 MB 350.1 kB/s eta 0:05:57
   ---------------------------------------- 0.1/124.9 MB 473.7 kB/s eta 0:04:24
   ---------------------------------------- 0.2/124.9 MB 655.6 kB/s eta 0:03:11
   ---------------------------------------- 0.3/124.9 MB 901.1 kB/s eta 0:02:19
   ---------------------------------------- 0.5/124.9 MB 1.1 MB/s eta 0:01:52
   ---------------------------------------- 0.8/124.9 MB 1.6 MB/s eta 0:01:17
   ---------------------------------------- 1.1/124.9 MB 2.1 MB/s eta 0

In [53]:
from xgboost import XGBClassifier

In [55]:
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
accuracy_score(y_test, y_pred)

0.737